# TAHAP 4 : **Program Pengujian pada Data Eksternal**

Program ini berfungsi untuk menguji akurasi sistem melalui pendeteksian data SIBI eksternal (diluar dari dataset training, test dan validation)dengan menggunakan file *best_lstm.pt* yang telah di-*export* dari program sebelumnya.

- Untuk menjalankan block program masing-masing, tekan *shift* + *enter* atau klik tombol icon '*Play*' ▶︎.
- <b>JALANKAN SETIAP BLOK PROGRAM SECARA SEKUENSIAL!</b>
- Konten di dalam program ini bisa digunakan sebagai panduan tutorial.

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --- Must match the training notebook model exactly ---
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, X, lengths):
        out, _ = self.lstm(X)  # (B,T,2H)
        B, T, D = out.shape

        # Mask padded timesteps
        mask = torch.arange(T, device=out.device).unsqueeze(0) < lengths.unsqueeze(1)
        mask_f = mask.float().unsqueeze(-1)

        summed = (out * mask_f).sum(dim=1)
        denom = lengths.clamp(min=1).unsqueeze(1).float()

        feat = summed / denom
        return self.classifier(feat)

def pad_or_truncate_with_length(sequence_2d: np.ndarray, target_len: int, n_features: int):
    T, F = sequence_2d.shape
    if F != n_features:
        raise ValueError(f"Expected {n_features} features, but got {F}")

    length = min(T, target_len)
    X = np.zeros((target_len, F), dtype=np.float32)
    X[:length, :] = sequence_2d[:length, :].astype(np.float32)
    return X, length


Device: cpu


In [5]:
def load_model_from_checkpoint(ckpt_path):
    ckpt = torch.load(ckpt_path, map_location="cpu")

    pad_len = int(ckpt["pad_len"])
    n_features = int(ckpt["n_features"])
    id_to_label = ckpt["id_to_label"]
    num_classes = len(id_to_label)

    model = BiLSTMClassifier(
        input_size=n_features,
        hidden_size=int(ckpt["hidden_size"]),
        num_layers=int(ckpt["num_layers"]),
        num_classes=num_classes,
        dropout=float(ckpt["dropout"])
    ).to(device)

    model.load_state_dict(ckpt["state_dict"])
    model.eval()

    return model, pad_len, n_features, id_to_label

def predict_csv(csv_path, model, pad_len, n_features, id_to_label, topk=5):
    df = pd.read_csv(csv_path)
    arr = df.values  # (T, 118)

    X_np, length = pad_or_truncate_with_length(arr, pad_len, n_features)

    x = torch.from_numpy(X_np).float().unsqueeze(0).to(device)  # (1,T,F)
    lengths = torch.tensor([length], dtype=torch.long).to(device)

    with torch.no_grad():
        logits = model(x, lengths)
        probs = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()

    topk = min(topk, len(probs))
    idxs = probs.argsort()[::-1][:topk]

    results = []
    for i in idxs:
        label = id_to_label[i] if isinstance(id_to_label, (list, tuple)) else id_to_label[int(i)]
        results.append((label, float(probs[i])))

    return results


Blok program dibawah berfungsi untuk menjalankan program-program di atas :

Untuk melakukan proses pengujian :
1. Import model yang telah di-*training* dan di-*export* dari folder yang disesuaikan (default = trained_model/best_lstm.pt)
2. Import data *.CSV* yang ingin diuji dari folder yang disesuaikan (default path/folder = csv (test data)/...)
3. Lihat hasil Top-5 prediction, lihat huruf dan akurasi yang tertinggi apakah menyesuaikan dengan nama objek yang diujikan.

In [ ]:
CKPT_PATH = "trained_model/best_lstm.pt"   # adjust if needed
CSV_PATH  = "csv (test data)/sentence40/a_075.csv"       # your external CSV

model, pad_len, n_features, id_to_label = load_model_from_checkpoint(CKPT_PATH)

preds = predict_csv(CSV_PATH, model, pad_len, n_features, id_to_label, topk=5)

print(f"External inference for: {Path(CSV_PATH).name}")
print("Top-5 prediction:")
for rank, (lab, p) in enumerate(preds, 1):
    print(f"  {rank}. {lab:>20s}  prob={p:.4f}")

External inference for: a_075.csv
Top-5 prediction:
  1.                    a  prob=0.9936
  2.                   an  prob=0.0024
  3.                    s  prob=0.0018
  4.                    y  prob=0.0017
  5.                    g  prob=0.0003
